In [1]:
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', None)  
from thesis_lib.data_processing import *
from thesis_lib.visual import *
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
from datetime import datetime
from collections import namedtuple

In [2]:
db = load_parquet('parquet_data')

Loading dataset:  laboratory
	 3086085
Loading dataset:  images
	 350403
Loading dataset:  hospitalizations
	 88256
Loading dataset:  surgeries
	 66750
Loading dataset:  sectors_admissions
	 154481
Loading dataset:  hospital_sectors
	 36
Formating integer columns
Formating date columns
Formating time columns
Formating datetime columns


In [20]:
class Patient:
    def __init__(self, patient_id):
        self.patient_id = patient_id

    def load_patient_data(self,db):
        
        hosp = db['hospitalizations']
        labo = db['laboratory']
        imag  = db['images']
        surg = db['surgeries']
        sect = db['sectors_admissions']
        
        
        self.hospitalizations_data = hosp[hosp['patient_id'] == self.patient_id].sort_values(
                                                                                by='admission_date')
        self.laboratory_data = labo[labo['patient_id'] == self.patient_id].sort_values(
                                                                                by=['admission_date',
                                                                                'labo_date'])
        self.images_data = imag[imag['patient_id'] == self.patient_id].sort_values(
                                                                                by=['admission_date',
                                                                                'image_date'])
        self.surgeries_data = surg[surg['patient_id'] == self.patient_id].sort_values(
                                                                                by=['admission_date',
                                                                                'surgery_date'])
        self.sectors_data = sect[sect['patient_id'] == self.patient_id].sort_values(
                                                                                by=['admission_id',
                                                                                'sector_admission_date'])
    
    @property
    def admission_history(self): 
        
        admission_key = namedtuple('admission_key', ['admission_number','admission_id'])
        
        if hasattr(self, 'hospitalizations_data'):
            return {admission_key(admission_number,admission_id) : Admission(admission_id, self) 
                   for admission_number, admission_id in enumerate(self.hospitalizations_data.admission_id)}
        
        else:
            print('Load patient data before')
    
    def get_historic_records(self):

        records_history = pd.DataFrame()
        
        for admission in self.admission_history:
            admission_records = self.admission_history[admission].get_admission_records()
            if records_history.empty:
                records_history = admission_records
            else:
                records_history  = records_history.append(admission_records, ignore_index=True)
        
        return records_history
            

In [28]:
def cast_list(list_to_cast):
    str_list = [str(i) for i in list_to_cast]
    
    return (',').join(str_list)
    

In [55]:
class Admission():
    def __init__(self, admission_id, patient):
        self.admission_id = admission_id
        
        self.load_admission_data(patient)
   

        
    def load_admission_data(self,patient): 
        
        self.admission_data = self.process_hospitalization_data(patient)
        
        self.discharge_datetime = datetime.strptime(str(self.admission_data['discharge_datetime']), 
                                                    '%Y-%m-%d %H:%M:%S')
        self.admission_datetime = datetime.strptime(str(self.admission_data['admission_datetime']),
                                                   '%Y-%m-%d %H:%M:%S')
        
        self.laboratory_data = self.process_laboratory_data(patient)                                         
        self.images_data = self.process_images_data(patient)
        self.surgeries_data = self.process_surgeries_data(patient)
        self.sectors_data = self.process_sectors_data(patient)

        
    def process_hospitalization_data(self, patient):
        
        return patient.hospitalizations_data[
                                            (patient.hospitalizations_data.admission_id == self.admission_id)
                                            ].to_dict(orient='records')[0]
        
    def process_laboratory_data(self, patient):
    
        return patient.laboratory_data[patient.laboratory_data.admission_id == self.admission_id]

    def process_images_data(self, patient):
        
        return patient.images_data[patient.images_data.admission_id == self.admission_id]
    
    def process_surgeries_data(self, patient):
        
        return patient.surgeries_data[patient.surgeries_data.admission_id == self.admission_id] 
    
    def process_sectors_data(self, patient):
        
        adm_sectors_data = patient.sectors_data[patient.sectors_data.admission_id == self.admission_id]
        adm_sectors_data['sector_stay'] = adm_sectors_data['sector_admission_datetime'
                                                          ].diff(periods=1).astype(
                                                                        'timedelta64[m]'
                                                                        ).shift(-1)  
        last_sector_stay = (self.discharge_datetime - 
                            adm_sectors_data.iloc[-1].sector_admission_datetime)
        
        
        adm_sectors_data.loc[adm_sectors_data.index[-1],
                                   'sector_stay'] = int(last_sector_stay.seconds/3600)
        
        return adm_sectors_data
    

    def get_labos_date_records(self,date):
        
        labos_records = {}
        
        mask = self.laboratory_data['labo_date'] == date
        labos = self.laboratory_data[mask]

        labos_records['labos_studies_names'] =  cast_list(labos.study_description.unique())
        labos_records['labos_set_count'] = labos.labo_id.nunique()
        labos_records['labos_count'] = len(labos.labo_id)
        labos_records['labos_requester_roles'] = cast_list(labos.requester_role.unique())
        labos_records['labos_requester_roles_count'] = labos.requester_role.nunique()
        labos_records['labos_emergencies'] = labos.emergency.sum()
        labos_records['labos_requesters'] = cast_list(labos.requester_name.unique())
        labos_records['labos_requesters_count'] = labos.requester_name.nunique()
        
        return labos_records 
    
    def get_images_date_records(self,date):
        
        images_records = {}

        mask = self.images_data['image_date'] == date
        images = self.images_data[mask]

        images_records['images_count'] =  images.image_id.nunique()
        images_records['images_study_types'] =  cast_list(images.type_of_service.unique())
        images_records['images_study_types_count'] =  images.type_of_service.nunique()
        images_records['images_studies_names'] = cast_list(images.study_description.unique())
        images_records['images_requesters'] = cast_list(images.requester_name.unique())
        images_records['images_requesters_count'] = images.requester_name.nunique()
        images_records['images_requester_roles'] = cast_list(images.requester_role.unique())
        images_records['images_requester_roles_count'] = images.requester_role.nunique()
        images_records['images_emergencies'] = images.emergency.sum()
    
        return images_records 
    
    def get_surgeries_date_records(self,date):
        
        surgeries_records = {}
        
        mask = self.surgeries_data['surgery_date'] == date
        surgeries = self.surgeries_data[mask]

        surgeries_records['surgeries_count'] =  len(surgeries.surgery_id.values)
        surgeries_records['surgeries_types'] =  cast_list(surgeries.surgery_type.values)
        surgeries_records['surgeries_types_count'] =  surgeries.surgery_type.nunique()
        surgeries_records['surgeries_scheduled'] =  cast_list(surgeries.scheduled_surgery.values)
        surgeries_records['surgeries_scheduled_done'] = cast_list(surgeries.scheduled_surgery_done.values)
        surgeries_records['surgeries_actual'] = cast_list(surgeries.actual_surgery.values)
        surgeries_records['surgeries_pre_surgery_duration'] = surgeries.pre_surgery_duration.sum()
        surgeries_records['surgeries_surgery_duration'] = surgeries.surgery_duration.sum()
        surgeries_records['surgeries_post_surgery_duration'] = surgeries.post_surgery_duration.sum()
        surgeries_records['surgeries_prep_duration'] = surgeries.surgery_prep_duration.sum()
        surgeries_records['surgeries_surgery_delay'] = surgeries.surgery_delay.sum()
        surgeries_records['surgeries_injury_condition'] = cast_list(surgeries.injury_condition.values)
        surgeries_records['surgeries_post_surgery_condition'] = cast_list(surgeries.post_surgery_condition.values)
        surgeries_records['surgeries_services'] = cast_list(surgeries.service_description.values)
        surgeries_records['surgeries_services_count'] =  surgeries.service_description.nunique()
        surgeries_records['surgeries_anesthesia_types'] =  cast_list(surgeries.anesthesia_type.values)
        surgeries_records['surgeries_bact_positive'] =  cast_list(surgeries.bact_positive.values)

        return surgeries_records 
    
    
    def get_date_sectors(self, date):
        
        if len(self.sectors_data) > 0:
            mask = self.sectors_data['sector_admission_date'] == date
            sectors = self.sectors_data[mask]

            if len(sectors) > 0:
                rv = sectors
            else: 
                rv = self.get_date_sectors(date-timedelta(days=1))
                
            return rv

        else:
            print('No sectors data available!')
            
    
    def get_sectors_date_records(self,date):
        
        sectors_records = {}
        sectors = self.get_date_sectors(date)
        
        sectors_records['sectors_last_sector'] = sectors.sector_code.values[-1]
        sectors_records['sectors_last_stay'] = sectors.sector_stay.values[-1]
        
        if sectors.iloc[-1].sector_admission_date == date:
            sectors_records['sectors_count'] = sectors.sector_code.nunique()
            sectors_records['sectors_names'] = cast_list(sectors.sector_code.values)
            sectors_records['sector_stay'] = cast_list(sectors.sector_stay.values)
        else:
        
            sectors_records['sectors_count'] = 0
            sectors_records['sectors_names'] = ''
            sectors_records['sector_stay'] = ''
    
        return sectors_records 
        
        
    def get_admission_records(self):
        
        length_of_stay = (self.admission_data['discharge_date']-
                          self.admission_data['admission_date']).days
        
        df = pd.DataFrame(columns=['patient_id',
                                   'admission_id', 
                                   'no._of_admission',
                                   'hosp_day_number', 
                                   'date',
                                   'discharge'])
        
        row = {'patient_id':patient.patient_id,
               'admission_id': self.admission_id,
               'no._of_admission': [key.admission_number for key in patient.admission_history.keys()if key.admission_id == self.admission_id][0],
               'labos_cumulative': 0,
               'images_cumulative': 0,
               'labos_set_cumulative':0,
               'surgeries_cumulative': 0}
          
        for day in range(length_of_stay):
            row['date'] = self.admission_data['admission_date'] + timedelta(day)
            row['hosp_day_number'] =  day
            row['discharge'] =  (True if day == (length_of_stay -1) else False)
            
            #Labos 
            labos_records = self.get_labos_date_records(row['date'])
            row.update(labos_records)
            row['labos_cumulative'] += row['labos_count']
            row['labos_set_cumulative'] += row['labos_set_count']
            
            #Images
            images_records = self.get_images_date_records(row['date'])
            row.update(images_records)
            row['images_cumulative'] += row['images_count']
            
            #Surgeries
            surgeries_records = self.get_surgeries_date_records(row['date'])
            row.update(surgeries_records)
            row['surgeries_cumulative'] += row['surgeries_count']
            
            #Sectors
            sectors_records = self.get_sectors_date_records(row['date'])
            row.update(sectors_records)
            
            df = df.append(row, ignore_index=True)

        return df

In [56]:
patient = Patient('3714610-6')

In [57]:
patient.load_patient_data(db)

In [58]:
patient.admission_history

{admission_key(admission_number=0, admission_id='466492-9'): <__main__.Admission at 0x112477128>}

In [59]:
patient.hospitalizations_data.admission_datetime

22   2017-01-01 14:03:00
Name: admission_datetime, dtype: datetime64[ns]

In [60]:
type(patient.admission_history[(0,'466492-9')].sectors_data.sector_code.nunique())

int

In [61]:
patient_clinic_history = patient.get_historic_records()

In [62]:
patient_clinic_history.head()

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge,images_count,images_cumulative,images_emergencies,images_requester_roles,images_requester_roles_count,images_requesters,images_requesters_count,images_studies_names,images_study_types,images_study_types_count,labos_count,labos_cumulative,labos_emergencies,labos_requester_roles,labos_requester_roles_count,labos_requesters,labos_requesters_count,labos_set_count,labos_set_cumulative,labos_studies_names,sector_stay,sectors_count,sectors_last_sector,sectors_last_stay,sectors_names,surgeries_actual,surgeries_anesthesia_types,surgeries_bact_positive,surgeries_count,surgeries_cumulative,surgeries_injury_condition,surgeries_post_surgery_condition,surgeries_post_surgery_duration,surgeries_pre_surgery_duration,surgeries_prep_duration,surgeries_scheduled,surgeries_scheduled_done,surgeries_services,surgeries_services_count,surgeries_surgery_delay,surgeries_surgery_duration,surgeries_types,surgeries_types_count
0,3714610-6,466492-9,0,0,2017-01-01,False,2.0,2.0,2.0,"MEDICO DE PISO,MEDICO CIRUGIA",2.0,"KIRCOS JESICA,SLUTZKY MAURICIO",2.0,"RX C/PORTATIL DE TORAX,TC DE CEREBRO SIN CONTR...","RAYOS,TOMOGRAFIA COMPUTADA (TAC)",2.0,7.0,7.0,7.0,MEDICO DE PISO,1.0,KIRCOS JESICA,1.0,1.0,1.0,"COAGULOGRAMA BASICO,GLUCOSA EN SANGRE,HEMOGRAM...","278.0,21370.0",2.0,UC1,21370.0,"EME,UC1",,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
1,3714610-6,466492-9,0,1,2017-01-02,False,0.0,2.0,0.0,,0.0,,0.0,,,0.0,16.0,23.0,7.0,MEDICO DE PISO,1.0,"BARRENA GONZALO,ESTRADA CATHERINE",2.0,3.0,4.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,UC1,21370.0,,"SUBDURAL COLECCION, CRANEOTOMIA C/EVACUACION",General,nan,1.0,1.0,Limpia,INT,9.0,46.0,25.0,"SUBDURAL COLECCION, CRANEOTOMIA C/EVACUACION",True,NEUROCIRUGIA,1.0,75.0,72.0,No Programada,1.0
2,3714610-6,466492-9,0,2,2017-01-03,False,1.0,3.0,0.0,AUDITORIA,1.0,BANEGA LAUREANA,1.0,TC DE CEREBRO SIN CONTRASTE,TOMOGRAFIA COMPUTADA (TAC),1.0,6.0,29.0,0.0,MEDICO DE PISO,1.0,BARRENA GONZALO,1.0,1.0,5.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,UC1,21370.0,,,,,0.0,1.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
3,3714610-6,466492-9,0,3,2017-01-04,False,0.0,3.0,0.0,,0.0,,0.0,,,0.0,6.0,35.0,0.0,MEDICO DE PISO,1.0,GAMBOA DURAN PAUL JOHN,1.0,1.0,6.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,UC1,21370.0,,,,,0.0,1.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
4,3714610-6,466492-9,0,4,2017-01-05,False,3.0,6.0,2.0,"MEDICO DE PISO,AUDITOR MEDICO,MEDICO DE UTI",3.0,"GAMBOA DURAN PAUL JOHN,MARSICANO DARIO,FIORI C...",3.0,"RX C/PORTATIL DE TORAX,TC DE CEREBRO SIN CONTR...","RAYOS,TOMOGRAFIA COMPUTADA (TAC)",2.0,9.0,44.0,0.0,MEDICO DE PISO,1.0,"ESTRADA CATHERINE,BARRENA GONZALO",2.0,2.0,8.0,"GLUCOSA EN SANGRE,IONOGRAMA EN SANGRE,GASES AR...",,0.0,UC1,21370.0,,,,,0.0,1.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0


In [63]:
patients = db['hospitalizations'].patient_id.unique()[:10]

In [64]:
patients


array(['3526141-4', '3714597-2', '3714595-2', '3672378-8', '3714598-2',
       '3690640-7', '2951027-4', '1394113-4', '3649342-5', '3420890-3'],
      dtype=object)

In [65]:
import pyarrow.parquet as pq
import pyarrow as pa


In [66]:
directory = 'patients_clinic_history'
try:
    os.mkdir(directory)
    print(directory, 'was created')
except:
    pass

print('Starting patients data processing...')


for patient_id in patients:
    patient = Patient(patient_id)
    patient.load_patient_data(db)
    patient_clinic_history = patient.get_historic_records()
    
    parquet = pa.Table.from_pandas(patient_clinic_history)
    pq.write_table(parquet, directory+'/patient_id_'+patient_id+'.parquet')
    
    print(patient_id+' clinic history saved')
    
    

Starting patients data processing...
3526141-4 clinic history saved
3714597-2 clinic history saved
3714595-2 clinic history saved
3672378-8 clinic history saved
3714598-2 clinic history saved
3690640-7 clinic history saved
2951027-4 clinic history saved
1394113-4 clinic history saved
3649342-5 clinic history saved
3420890-3 clinic history saved


In [71]:
path= directory+'/'
patients_clinic_history = []
for r, d, f in os.walk(path):
    for file in f:
        if '.parquet' in file:
            df_name = file[:-8]
            print('Loading clinic history patient #:',df_name)
            patients_clinic_history.append(pd.read_parquet(path+file))
    
    

Loading clinic history patient #: patient_id_2951027-4
Loading clinic history patient #: patient_id_3672378-8
Loading clinic history patient #: patient_id_3690640-7
Loading clinic history patient #: patient_id_3526141-4
Loading clinic history patient #: patient_id_3714595-2
Loading clinic history patient #: patient_id_3649342-5
Loading clinic history patient #: patient_id_3714598-2
Loading clinic history patient #: patient_id_1394113-4
Loading clinic history patient #: patient_id_3420890-3
Loading clinic history patient #: patient_id_3714597-2


In [73]:
patients_clinic_history [0]

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge,images_count,images_cumulative,images_emergencies,images_requester_roles,images_requester_roles_count,images_requesters,images_requesters_count,images_studies_names,images_study_types,images_study_types_count,labos_count,labos_cumulative,labos_emergencies,labos_requester_roles,labos_requester_roles_count,labos_requesters,labos_requesters_count,labos_set_count,labos_set_cumulative,labos_studies_names,sector_stay,sectors_count,sectors_last_sector,sectors_last_stay,sectors_names,surgeries_actual,surgeries_anesthesia_types,surgeries_bact_positive,surgeries_count,surgeries_cumulative,surgeries_injury_condition,surgeries_post_surgery_condition,surgeries_post_surgery_duration,surgeries_pre_surgery_duration,surgeries_prep_duration,surgeries_scheduled,surgeries_scheduled_done,surgeries_services,surgeries_services_count,surgeries_surgery_delay,surgeries_surgery_duration,surgeries_types,surgeries_types_count
0,2951027-4,466476-9,0,0,2017-01-01,False,0.0,0.0,0.0,,0.0,,0.0,,,0.0,6.0,6.0,0.0,MEDICO DE PISO,1.0,ALDAO SOFIA,1.0,1.0,1.0,"COAGULOGRAMA BASICO,GLUCOSA EN SANGRE,HEMOGRAM...",11.0,1.0,T12,11.0,T12,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
1,2951027-4,466476-9,0,1,2017-01-02,False,0.0,0.0,0.0,,0.0,,0.0,,,0.0,10.0,16.0,0.0,MEDICO DE PISO,1.0,"CARDOSO PAULA,MALDONADO FEDERICO",2.0,2.0,3.0,"COAGULOGRAMA BASICO,HEMOGRAMA,IONOGRAMA EN SAN...",,0.0,T12,11.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
2,2951027-4,466476-9,0,2,2017-01-03,True,0.0,0.0,0.0,,0.0,,0.0,,,0.0,0.0,16.0,0.0,,0.0,,0.0,0.0,3.0,,,0.0,T12,11.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0


In [74]:
patients_clinic_history[1]

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge,images_count,images_cumulative,images_emergencies,images_requester_roles,images_requester_roles_count,images_requesters,images_requesters_count,images_studies_names,images_study_types,images_study_types_count,labos_count,labos_cumulative,labos_emergencies,labos_requester_roles,labos_requester_roles_count,labos_requesters,labos_requesters_count,labos_set_count,labos_set_cumulative,labos_studies_names,sector_stay,sectors_count,sectors_last_sector,sectors_last_stay,sectors_names,surgeries_actual,surgeries_anesthesia_types,surgeries_bact_positive,surgeries_count,surgeries_cumulative,surgeries_injury_condition,surgeries_post_surgery_condition,surgeries_post_surgery_duration,surgeries_pre_surgery_duration,surgeries_prep_duration,surgeries_scheduled,surgeries_scheduled_done,surgeries_services,surgeries_services_count,surgeries_surgery_delay,surgeries_surgery_duration,surgeries_types,surgeries_types_count
0,3672378-8,466473-0,0,0,2017-01-01,False,5.0,5.0,7.0,"MEDICO DE UCI,MEDICO DE PISO",2.0,"CANIS DIEGO,GRANJA SANTIAGO,PUENTE KAROLA ADRIANA",3.0,"ABDOMEN FRENTE, DE PIE Y ACOSTADO. RX DE,RX C/...","RAYOS,ECOGRAFIA,TOMOGRAFIA COMPUTADA (TAC)",3.0,23.0,23.0,16.0,"MEDICO DE UCI,MEDICO DE PISO",2.0,"CANIS DIEGO,CALIVA ERNESTO ANDRES",2.0,2.0,2.0,"AMILASA EN SANGRE,ALBUMINA EN SANGRE,COAGULOGR...","290.0,29.0,329.0,1.0",2.0,T12,1.0,"EME,T12,EME,T12",,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
1,3672378-8,466473-0,0,1,2017-01-02,False,0.0,5.0,0.0,,0.0,,0.0,,,0.0,5.0,28.0,0.0,MEDICO DE PISO,1.0,CHAVEZ JORGE,1.0,1.0,3.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,T12,1.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
2,3672378-8,466473-0,0,2,2017-01-03,True,0.0,5.0,0.0,,0.0,,0.0,,,0.0,0.0,28.0,0.0,,0.0,,0.0,0.0,3.0,,,0.0,T12,1.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0


In [75]:
patients_clinic_history[3]

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge,images_count,images_cumulative,images_emergencies,images_requester_roles,images_requester_roles_count,images_requesters,images_requesters_count,images_studies_names,images_study_types,images_study_types_count,labos_count,labos_cumulative,labos_emergencies,labos_requester_roles,labos_requester_roles_count,labos_requesters,labos_requesters_count,labos_set_count,labos_set_cumulative,labos_studies_names,sector_stay,sectors_count,sectors_last_sector,sectors_last_stay,sectors_names,surgeries_actual,surgeries_anesthesia_types,surgeries_bact_positive,surgeries_count,surgeries_cumulative,surgeries_injury_condition,surgeries_post_surgery_condition,surgeries_post_surgery_duration,surgeries_pre_surgery_duration,surgeries_prep_duration,surgeries_scheduled,surgeries_scheduled_done,surgeries_services,surgeries_services_count,surgeries_surgery_delay,surgeries_surgery_duration,surgeries_types,surgeries_types_count
0,3526141-4,466470-1,0,0,2017-01-01,False,1.0,1.0,1.0,MEDICO DE PISO,1.0,VEDIA ANICETA,1.0,RX C/PORTATIL DE TORAX,RAYOS,1.0,10.0,10.0,10.0,MEDICO DE PISO,1.0,VEDIA ANICETA,1.0,2.0,2.0,"COAGULOGRAMA BASICO,GLUCOSA EN SANGRE,HEMOGRAM...","353.0,10.0",2.0,T16,10.0,"EME,T16",,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
1,3526141-4,466470-1,0,1,2017-01-02,False,2.0,3.0,2.0,"MEDICO DE PISO,AUDITORIA MEDICA",2.0,"GAGLIARDONE LORENA DAIANA,DESIDERIO MARIA EMILIA",2.0,"RODILLA, RX DE - (2 EXPOSICIONES x PLACA),TC D...","RAYOS,TOMOGRAFIA COMPUTADA (TAC)",2.0,17.0,27.0,9.0,MEDICO DE PISO,1.0,GAGLIARDONE LORENA DAIANA,1.0,3.0,5.0,"CALCIO IONICO,COAGULOGRAMA BASICO,FOSFORO EN S...",,0.0,T16,10.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
2,3526141-4,466470-1,0,2,2017-01-03,True,0.0,3.0,0.0,,0.0,,0.0,,,0.0,5.0,32.0,0.0,MEDICO RESIDENTE,1.0,BRUTOMESO MARIA LAURA,1.0,1.0,6.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,T16,10.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
3,3526141-4,500538-7,1,0,2018-02-26,False,2.0,2.0,1.0,MEDICO DE PISO,1.0,"ROJAS MARIANA GISELLE,PALOMINO ROMERO LUIS CARLOS",2.0,"RX C/PORTATIL DE TORAX,ECO-DOPPLER COLOR CARDIACO","RAYOS,ECOCARDIOGRAMA",2.0,19.0,19.0,9.0,MEDICO DE PISO,1.0,"ROJAS MARIANA GISELLE,PALOMINO ROMERO LUIS CARLOS",2.0,3.0,3.0,"COAGULOGRAMA BASICO,GLUCOSA EN SANGRE,HEMOGRAM...","345.0,20.0",2.0,T11,20.0,"EME,T11",,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
4,3526141-4,500538-7,1,1,2018-02-27,False,0.0,2.0,0.0,,0.0,,0.0,,,0.0,7.0,26.0,0.0,MEDICA DE PISO,1.0,ROCCA LUCIANA,1.0,1.0,4.0,"GLUCOSA EN SANGRE,HEMOGRAMA,IONOGRAMA EN SANGR...",,0.0,T11,20.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
5,3526141-4,500538-7,1,2,2018-02-28,False,0.0,2.0,0.0,,0.0,,0.0,,,0.0,0.0,26.0,0.0,,0.0,,0.0,0.0,4.0,,,0.0,T11,20.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
6,3526141-4,500538-7,1,3,2018-03-01,True,2.0,4.0,0.0,MEDICA DE PISO,1.0,ROCCA LUCIANA,1.0,"TORAX FRENTE, RX DE (C/S TRAZ-RELL),ECOGRAFIA ...","RAYOS,ECOGRAFIA",2.0,0.0,26.0,0.0,,0.0,,0.0,0.0,4.0,,,0.0,T11,20.0,,,,,0.0,0.0,,,0.0,0.0,0.0,,,,0.0,0.0,0.0,,0.0
